In [1]:
from os import path
import os
import glob
import platform
import shutil
import tempfile
import sqlite3
import re
import json

In [2]:
class Aggregator:
    def __enter__(self):
        # scan browsers internal history dbs
        curr_os = platform.system()
        home_dir = path.expanduser('~')
        oss_bin_paths = {
            'Linux'  : ('/', 'usr', 'bin'),
            'Darwin' : ('/', 'Applications'),
            'Windows': ('C:/', 'Program Files')
        }
        # /usr/bin: symlinked from /usr/lib
        install_dirs = {
            os : path.join(*path_comps) 
            for os, path_comps in oss_bin_paths.items()
        }
        browsers_data_stores = {
            'Chrome' : ('.config', 'google-chrome', 'Default', 'History'),
            'Firefox' : ('.mozilla', 'firefox', '*.default', 'places.sqlite')
        }
        browsers = {
            browser : path.join(home_dir, *path_comps) 
            for browser, path_comps in browsers_data_stores.items()
        }        
        # store both locked and lock-free db copies
        self.db_files = {
            browser : {
                file_t : {} for file_t in ['orig', 'tmp']
            }
            for browser in browsers
        }
        
        for browser in browsers:
            found = glob.glob(f'{install_dirs[curr_os]}/*{browser.lower()}*') is not None
            if found:
                orig_file = glob.glob(browsers[browser])
                tmp_file = self._tmp_copy(*orig_file)
                self.db_files[browser]['orig'] = orig_file
                self.db_files[browser]['tmp'] = tmp_file
                
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # delete tmp copies on exit
        for _, files in self.db_files.items():
            os.remove(files['tmp'])
                    
    def _tmp_copy(self, original_file):
        tmp = tempfile.gettempdir()
        filename = path.basename(original_file)
        tmp_file = path.join(tmp, filename)
        shutil.copy2(original_file, tmp_file)
        return tmp_file
        
    @classmethod
    def _regexp(cls, regex, field_val):
        # base REGEXP implementation
        return bool(re.search(regex, field_val))
    
    def _extract(self, row):
        return row[0]
    
    def _get_history_tables(self, conn, cursor):
        conn.create_function("REGEXP", 2, Aggregator._regexp)
        regex = '.*(history|visit).*'
        query = cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type="table" and name REGEXP ?
        """, [regex])
        rows = query.fetchall()
        rows = [*map(self._extract, rows)]
        return rows
   
    def _get_fields(self, table):
        # looksup a table's columns
        query = self.cursor.execute(f'SELECT * FROM {table}')
        fields = [*map(self._extract, query.description)]
        return fields
            
    def _to_dict(self, row, fields):
        # converts row to dict by mapping columns to values
        dict_ = {}
        for i, val in enumerate(row):
            field = self._extract(fields[i])
            if field == 'typed':
                val = self._typed(val)
            elif field == 'visit_type':
                val = self._visit_type(val)
                val = val.split('_')[-1].lower()
            dict_[field] = val
        return dict_
    
    def _to_json(self, cursor):
        fields = cursor.description
        rows = cursor.fetchall()
        rows = [ self._to_dict(row, fields) for row in rows ]
        rows_as_json = json.dumps(rows, indent = 2)
        return rows_as_json
    
    def normalise(self, history_files):
        # normalise history fields across browsers
        raise NotImplementedError
                                 
    def get_history_as_json(self):
        # returns browser history as list of json documents
        raise NotImplementedError

In [3]:
class AggregatorFirefox(Aggregator):
    def __init__(self, db_file):
        # raises sqlite3.OperationalError: unable to open database file
        self.conn = sqlite3.connect(f'file:{db_file}?mode=ro', uri = True)
        self.cursor = self.conn.cursor()
        sqlite3.enable_callback_tracebacks(True)
            
    def _typed(self, boolean):
        # maps typed status to their written rep for ease of querying
        return ['no', 'yes'][boolean]

    def _visit_type(self, enum):
        # maps visit types to their written rep for ease of querying
        return [
            'TRANSITION_LINK',
            'TRANSITION_TYPED',
            'TRANSITION_BOOKMARK',
            'TRANSITION_EMBED',
            'TRANSITION_REDIRECT_PERMANENT',
            'TRANSITION_REDIRECT_TEMPORARY',
            'TRANSITION_DOWNLOAD',
            'TRANSITION_FRAMED_LINK',
            'TRANSITION_RELOAD'
        ][enum - 1]

    def get_history_as_json(self):
        # returns browser history as list of json documents
        self.cursor.execute("""
            SELECT moz_historyvisits.id, 
               moz_places.url, 
               moz_places.title, 
               moz_places.visit_count, 
               moz_places.typed, 
               DATETIME(moz_historyvisits.visit_date/1000000, 'unixepoch') as visit_date, 
               moz_historyvisits.visit_type 
            FROM moz_places, moz_historyvisits 
            WHERE moz_historyvisits.place_id = moz_places.id;
        """)
        rows_as_json = self._to_json(self.cursor)
        return rows_as_json

In [4]:
class AggregatorChrome(Aggregator):
    def __init__(self, db_file):
        # raises sqlite3.OperationalError: unable to open database file
        self.conn = sqlite3.connect(f'file:{db_file}?mode=ro', uri = True)
        self.cursor = self.conn.cursor()
        sqlite3.enable_callback_tracebacks(True)
         
    def _typed(self, boolean):
        # maps typed status to their written rep for ease of querying
        return ['no', 'yes'][boolean]

    def _visit_type(self, enum):
        # maps visit types to their written rep for ease of querying
        CORE_MASK = 0xFF
        return [
            'LINK',
            'TYPED',
            'AUTO_BOOKMARK',
            'AUTO_SUBFRAME',
            'MANUAL_SUBFRAME',
            'GENERATED',
            'START_PAGE',
            'FORM_SUBMIT',
            'RELOAD',
            'KEYWORD',
            'KEYWORD_GENERATED'
        ][enum & CORE_MASK]

    def get_history_as_json(self):
        # returns browser history as list of json documents
        self.cursor.execute("""
            SELECT urls.url, 
                   urls.title, 
                   urls.visit_count, 
                   urls.typed_count, 
                   urls.last_visit_time, 
                   urls.hidden, 
                   DATETIME(visit_time / 1000000 + (strftime('%s', '1601-01-01')), 'unixepoch', 'localtime') as visit_date, 
                   visits.from_visit, 
                   visits.transition as visit_type
            FROM urls, visits
            WHERE
                urls.id = visits.url
        """)
        rows_as_json = self._to_json(self.cursor)
        return rows_as_json

In [5]:
with Aggregator() as agg:
    db_files = agg.db_files
    for browser, AggregatorBrowser in list(zip(db_files, [AggregatorChrome, AggregatorFirefox])):
        db_file = db_files[browser]['tmp']
        print(f'\n{browser}: ', db_file, end="\n\n")
        try:
            agg_browser = AggregatorBrowser(db_file) 
            tables = agg_browser._get_history_tables(agg_browser.conn, agg_browser.cursor)
            tables

            for table in tables:
                fields = agg_browser._get_fields(table)
                print(table, fields, sep = "\n", end = "\n\n")

            json_docs = agg_browser.get_history_as_json()
            print(json_docs)
        except sqlite3.OperationalError as e:
            print(str(e))


Chrome:  /tmp/History

visits
['id', 'url', 'visit_time', 'from_visit', 'transition', 'segment_id', 'visit_duration', 'incremented_omnibox_typed_score', 'opener_visit', 'originator_cache_guid', 'originator_visit_id', 'originator_from_visit', 'originator_opener_visit', 'is_known_to_sync']

visit_source
['id', 'source']

clusters_and_visits
['cluster_id', 'visit_id', 'score', 'engagement_score', 'url_for_deduping', 'normalized_url', 'url_for_display']

cluster_visit_duplicates
['visit_id', 'duplicate_visit_id']

[
  {
    "url": "https://www.google.com/search?q=chrome+history+sqlite&oq=chrome+history+sqlite&aqs=chrome..69i57j0i512l2j0i22i30l7.3026j0j7&sourceid=chrome&ie=UTF-8",
    "title": "chrome history sqlite - Google Search",
    "visit_count": 2,
    "typed_count": 0,
    "last_visit_time": 13323720168788762,
    "hidden": 0,
    "visit_date": "2023-03-19 17:22:48",
    "from_visit": 0,
    "visit_type": "generated"
  },
  {
    "url": "https://www.google.com/search?q=chrome+histo

### Firefox

Note: You have to copy `places.sqlite` file from the default location (at `/home/user/.mozilla/firefox/*.default/places.sqlite'`) as Firefox places a lock on the file for preventing any edits (see: [editing](https://kb.mozillazine.org/Places.sqlite#:~:text=The%20file%20%22places.,a%20record%20of%20visited%20pages).) 
#### Browser history tables
See: [Firefox/Browsing history database](https://en.wikiversity.org/wiki/Firefox/Browsing_history_database)
There are three tables of interest, i.e. (1) moz_historyvisits, (2) moz_inputhistory and (3) moz_places.

For a visual representation of the schemas, see: [here](https://wiki.mozilla.org/images/d/d5/Places.sqlite.schema3.pdf)

(1) Contais the web history; has the following fields: 
```
id
from_visit
place_id
visit_date
visit_type
session
source
triggeringPlaceId
```

These are the history visit [visit_types](https://searchfox.org/mozilla-esr60/source/toolkit/components/places/nsINavHistoryService.idl#1185)

```
TRANSITION_LINK = 1
TRANSITION_TYPED = 2
TRANSITION_BOOKMARK	 = 3
TRANSITION_EMBED = 4
TRANSITION_REDIRECT_PERMANENT = 5
TRANSITION_REDIRECT_TEMPORARY = 6
TRANSITION_DOWNLOAD = 7
TRANSITION_FRAMED_LINK = 8
TRANSITION_RELOAD = 9
```


(2) Contains URLs the user typed in; has the following fields: 
```
place_id
input
use_count
```

(3) Contains various bits of info about visited URLs; has the following fields
```
id
url
title
rev_host
URLs
hidden
typed
frecency
last_visit_date
guid
foreign_count
url_hash
description
preview_image_url
origin_id
site_name
recalc_frecency
```